In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [1]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [1]:
train.head()

**EXPLORATORY DATA ANALYSIS - Initial Investigation to understand the Data**

In [1]:
print(train.dtypes)

In [1]:
train.shape

In [1]:
train.columns

In [1]:
print(train.nunique())

In [1]:
train.describe()

In [1]:
train.info()

DATA UNDERSTANDING

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt

In [1]:
numeric_variables = ["PassengerId", "Pclass", "Survived", "Age", "SibSp", "Parch", "Fare"]
categorical_variables = ["Name" , "Sex", "Ticket", "Embarked"]

In [1]:
sns.barplot(x = "Pclass" , y = "Survived" , data = train)
#Passenger class 1 people has survived more

In [1]:
Pclass1=train["Survived"][train["Pclass"] == 1].value_counts(normalize=True)[1]*100
Pclass2=train["Survived"][train["Pclass"] == 2].value_counts(normalize=True)[1]*100
Pclass3=train["Survived"][train["Pclass"] == 3].value_counts(normalize=True)[1]*100
Pclass_Values=[Pclass1,Pclass2,Pclass3]
plt.pie(Pclass_Values, labels=('Pclass1','Pclass2','Pclass3') ,explode=(0.1,0.0,0.0), autopct='%1.1f%%')

In [1]:
sns.barplot(x = "Sex" , y = "Survived" , data = train)
#females has survived more

In [1]:
Females=train["Survived"][train["Sex"] == 'female'].value_counts(normalize=True)[1]*100
males=train["Survived"][train["Sex"] == 'male'].value_counts(normalize=True)[1]*100
Sex_values=[males,Females]
plt.pie(Sex_values, labels=('males','Females'),explode=(0.0,0.1),autopct='%1.1f%%')

In [1]:
sns.barplot(x = "SibSp" , y = "Survived" , data = train)
#people with 1 siblings survived more

In [1]:
a = train["Survived"][train['SibSp'] == 0].value_counts(normalize=True)[1]*100
b = train["Survived"][train['SibSp'] == 1].value_counts(normalize=True)[1]*100
c = train["Survived"][train['SibSp'] == 2].value_counts(normalize=True)[1]*100
d = train["Survived"][train['SibSp'] == 3].value_counts(normalize=True)[1]*100
e = train["Survived"][train['SibSp'] == 4].value_counts(normalize=True)[1]*100
Sibling_values=[a,b,c,d,e]
plt.pie(Sibling_values, labels=('SibSp0','SibSp1','SibSp2','SibSp3','SibSp4'),explode=(0.1,0.1,0.1,0.1,0.1), autopct='%1.1f%%')

In [1]:
sns.barplot(x = "Parch" , y = "Survived" , data = train)
#people with parch = 3 survived more

In [1]:
P0=train["Survived"][train["Parch"] == 0].value_counts(normalize=True)[1]*100
P1=train["Survived"][train["Parch"] == 1].value_counts(normalize=True)[1]*100
P2=train["Survived"][train["Parch"] == 2].value_counts(normalize=True)[1]*100
P3=train["Survived"][train["Parch"] == 3].value_counts(normalize=True)[1]*100
P5=train["Survived"][train["Parch"] == 5].value_counts(normalize=True)[1]*100
Parch_values=[P0,P1,P2,P3,P5]
plt.pie(Parch_values, labels=('Parch0','Parch1','Parch2','Parch3','Parch5'),explode=(0.0,0.0,0.0,0.1,0.0),autopct='%1.1f%%')

MISSING VALUE ANALYSIS

In [1]:
train.isnull().sum()

In [1]:
missing_value_percentage = print(train.isnull().sum()/len(train))
missing_value_percentage

drop cabin as it has a huge number of missing values and few other variables which are not important for the model

In [1]:
train = train.drop(["Cabin", "Name", "Fare", "Ticket"], axis = 1)

In [1]:
train.head()

In [1]:
print(train.isnull().sum())

#lets impute other missing values 

In [1]:
#impute age with mean as it is numeric 
#impute embarked with mode as it is categorical
train['Age'] = train['Age'].fillna(train['Age'].mean())
train["Embarked"] = pd.Categorical(train["Embarked"])
train["Embarked"] = train["Embarked"].cat.codes
train["Embarked"] = train["Embarked"].fillna(train["Embarked"].mode())
print(train.isnull().sum())


In [1]:
train["Embarked"] = train["Embarked"].astype("object")
print(train.dtypes)

In [1]:
train.head()

In [1]:
train["Sex"] = pd.Categorical(train["Sex"])
train["Sex"] = train["Sex"].cat.codes
train["Sex"] = train["Sex"].astype("object")

#male = 1, female = 0

In [1]:
train.head()

In [1]:
train.shape

In [1]:
train.describe()

Outlier analysis

In [1]:
numeric_variables2 = ["Pclass", "Survived", "Age",]
categorical_variables2 = [ "Sex", "Embarked"]

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt

for i in numeric_variables2 :
    print(i)
    sns.boxplot(y = train[i])
    plt.xlabel(i)
    plt.ylabel("Values")
    plt.title("Boxplot of " + i)
    plt.show()

**there is lot of outliers in the data so lets replace those outliers with NA and impute it **

In [1]:
# Identify outliers
#calculate Inner Fence, Outer Fence, and IQR

for i in numeric_variables2:
    print(i)
    q75, q25 = np.percentile(train.loc[:,i], [75, 25])
    iqr = q75 - q25
    Innerfence = q25 - (iqr*1.5)
    Upperfence = q75 + (iqr*1.5)
    print("Innerfence= "+str(Innerfence))
    print("Upperfence= "+str(Upperfence)) 
    print("IQR ="+str(iqr))
    

# replace outliers with NA

    train.loc[train[i]<Innerfence, i] = np.nan
    train.loc[train[i]>Upperfence, i] = np.nan

In [1]:
print(train.isnull().sum()/len(train))

In [1]:
#impute age with mean
train['Age'] = train['Age'].fillna(train['Age'].mean())
print(train.isnull().sum())        

In [1]:
train.head()

In [1]:
train.dtypes

**FEATURE SELECTION**

In [1]:
numeric_variables3 = ["PassengerId", "Survived", "Pclass", "Age", "SibsSp", "Parch"]

**CORRELATION ANALYSIS**

In [1]:
Correlation = train.loc[:, numeric_variables3]
correlation_result = Correlation.corr()
print(correlation_result)
    

In [1]:
heatmap =  sns.heatmap(correlation_result)

#No collinearity is found

In [1]:
#Data Distribution

x = train.drop(["Survived"], axis = 1)
y = train["Survived"]

In [1]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 

#divide the data into train and test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.20, random_state=0)

MODELLING

RANDOM FOREST

In [1]:
from sklearn.ensemble import RandomForestClassifier

RM_model = RandomForestClassifier(n_jobs = 100, n_estimators = 100, random_state = 123)
RM_model.fit(x_train, y_train)

In [1]:
RM_model.score(x_test, y_test)

In [1]:
y_predrm = RM_model.predict(x_test)

#Accuracy

accuracy_rm = round(accuracy_score(y_predrm, y_test)*100,2)
print(accuracy_rm)

In [1]:
from sklearn.metrics import classification_report  
print(classification_report(y_predrm, y_test))

In [1]:
from sklearn.metrics import confusion_matrix 
print(confusion_matrix(y_predrm, y_test))

LOGISTIC REGRESSION

In [1]:
from sklearn.linear_model import LogisticRegression
LR_Model = LogisticRegression()
LR_Model.fit(x_train, y_train)

In [1]:
LR_Model.score(x_test,y_test)

In [1]:
y_predlr = LR_Model.predict(x_test)

#Accuracy
accuracy_lr = round(accuracy_score(y_predlr, y_test) * 100, 2)
print("Accuracy:",accuracy_lr)

In [1]:
print(classification_report(y_predlr, y_test))

In [1]:
print(confusion_matrix(y_predlr, y_test))

It is found that in normal cases Random Forest is a better model for the given dataset, hypertuning can be done to improve the accuracy far more better

Predict in Test data

In [1]:
test.head()

In [1]:
#Drop unnecessary columns 
test = test.drop(['Name','Ticket','Fare','Cabin'], axis=1)
test.head()

In [1]:
#Convert datatype
test["Sex"] = pd.Categorical(test["Sex"])
test["Sex"] = test["Sex"].cat.codes
test["Sex"] = test["Sex"].astype("object")

test["Embarked"] = pd.Categorical(test["Embarked"])
test["Embarked"] = test["Embarked"].cat.codes
test["Embarked"] = test["Embarked"].astype("object")

test.head()



In [1]:
#Check for NA values
test.isnull().sum()

In [1]:
test["Age"] = test["Age"].fillna(test["Age"].mean())

In [1]:
test.isnull().sum()

In [1]:
test.head()

In [1]:
test["Survived"] = RM_model.predict(test)

In [1]:
test.head()

In [1]:
predicted_test = test.drop(['Pclass','Sex','Age','SibSp', "Parch", "Embarked"], axis=1)


In [1]:
y_output = predicted_test["Survived"]


In [1]:
given_output = pd.read_csv("../input/gender_submission.csv")

In [1]:
y_given= given_output["Survived"]

In [1]:
accuracy_final = round(accuracy_score(y_output, y_given) * 100, 2)
print("Accuracy:",accuracy_final)

In [1]:
print(confusion_matrix(y_output, y_given))

In [1]:
sns.countplot(x="Survived",data=test , )

In [1]:
test['Survived'].value_counts()

got 77.03 % accuracy with resembelence to the actual output, the model can be hypertuned , variable engineered etc can be done to improve the accuracy of the model 

In [1]:
predicted_test.to_csv('submission.csv', index=False)